# library loading

In [1]:
#!pip install ko-sentence-transformers 
from sentence_transformers import SentenceTransformer, models
from ko_sentence_transformers.models import KoBertTransformer

# Model loading
word_embedding_model = KoBertTransformer("monologg/kobert", max_seq_length=75)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), pooling_mode='mean')
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

from sentence_transformers import SentenceTransformer, util
import numpy as np
import pandas as pd

# embedder loading
embedder = SentenceTransformer("jhgan/ko-sbert-sts")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


# data loading

In [2]:
# 데이터 불러오기
df = pd.read_csv('DBdata.csv')  # 경로 수정

In [3]:
## df랑 모듈들 저장 가능하냐?
import torch
torch.save({'df':df, 'embedder':embedder, 'word_embedding_model':word_embedding_model,\
            'pooling_model':pooling_model, 'model':model},"model.tar")

# 기본 수행

In [33]:
# 전체 카테고리(tot)
## corpus
texts = ''
for line in df.rv[:]:
    texts = texts + line
text = list(filter(None, texts.split(sep='\n')))
## embedding
tot_corpus = text
tot_embedding = embedder.encode(tot_corpus, convert_to_tensor = True)

In [34]:
# 기초케어 카테고리(bas)
## 범위 split
fir = df[df['cate'] == '토너'].index[0]
las = df[df['cate'] == '미스트'].index[-1]
## corpus
texts = ''
for line in df.rv[fir:las]:
    texts = texts + line
text = list(filter(None, texts.split(sep='\n')))

## embedding
bas_corpus = text
bas_embedding = embedder.encode(bas_corpus, convert_to_tensor = True)

In [3]:
# 색조케어 카테고리(col)
## 범위 split
fir = df[df['cate'] == '립'].index[0]
las = df[df['cate'] == '아이'].index[-1]
## corpus
texts = ''
for line in df.rv[fir:las]:
    texts = texts + line
text = list(filter(None, texts.split(sep='\n')))

## embedding
col_corpus = text
col_embedding = embedder.encode(col_corpus, convert_to_tensor = True)

In [36]:
# toch 파일로 저장
import torch
torch.save({'tot_corpus':tot_corpus, 'tot_embedding':tot_embedding, 'bas_corpus':bas_corpus,\
            'bas_embedding':bas_embedding, 'col_corpus':col_corpus, 'col_embedding':col_embedding},"check_point.tar")

# 필요한 작업 수행

In [8]:
# 출력폼 함수
def menu(no):
    print('-'*30)
    print('★원하는 카테고리를 선택해주세요★')
    print('1. 전체')
    print('2. 기초')
    print('3. 색조')
    print('-'*30)
    embed, corpus, cate = choice_cate(no)
    return embed, corpus, cate 
    
    
# 카테고리 선택 함수
def choice_cate(no):
    num = no
    if num == 1:
        embed = tot_embedding
        corpus = tot_corpus
        cate = '전체'
    elif num == 2:
        embed = bas_embedding
        corpus = bas_corpus
        cate = '기초'
    elif num == 3:
        embed = col_embedding
        corpus = col_corpus
        cate = '색조'
    else:
        print('잘못된 선택입니다. 1~3 사이의 숫자로 입력해주세요.')
        return menu()
    return embed, corpus, cate
        

# 유사 문장 리턴함수
def num_return(text, embed, corpus, cate):
    query= text # str(input('검색하기 : '))
    cate_embedding = embed
    # 유사도 평가를 위한 작업
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    cos_scores = util.pytorch_cos_sim(query_embedding, cate_embedding)[0]
    cos_scores = cos_scores.cpu()
    # 상위 결과 추출
    top_results = np.argpartition(-cos_scores, range(5))[0:5]
    print("\n▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽\n")
    print("요청 문장:", query)
    print(f"🌼{cate} 카테고리의 리뷰들 중 가장 유사한 5개 리뷰문장🌼\n")
    for v, idx in enumerate(top_results[0:5]):
        text = corpus[idx].strip() 
        for i in range(3344):
            if text in df['rv'][i]:
                pname = df['pname'][i]
        print(v+1, pname, '\n','▶',text, "(Score: %.4f)" % (cos_scores[idx]))
    print("\n△△△△△△△△△△△△△△△△△△\n")
    
def play_rv(no, text):
    embed, corpus, cate = menu(no)
    num_return(text, embed, corpus, cate)

In [21]:
def list_return(text, embed, corpus, cate):
    global answer
    answer = []
    query= text # str(input('검색하기 : '))
    cate_embedding = embed
    # 유사도 평가를 위한 작업
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    cos_scores = util.pytorch_cos_sim(query_embedding, cate_embedding)[0]
    cos_scores = cos_scores.cpu()
    # 상위 결과 추출
    top_results = np.argpartition(-cos_scores, range(5))[0:5]
    print("\n▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽\n")
    print("요청 문장:", query)
    print(f"🌼{cate} 카테고리의 리뷰들 중 가장 유사한 5개 리뷰문장🌼\n")
    for v, idx in enumerate(top_results[0:5]):
        text = corpus[idx].strip() 
        for i in range(3344):
            if text in df['rv'][i]:
                pname = df['pname'][i]
        answer.append([v+1, pname, text, np.round(cos_scores[idx],4)])
#         print(answer)
        print(v+1, pname, '\n','▶',text, "(Score: %.4f)" % (cos_scores[idx]))
    print("\n△△△△△△△△△△△△△△△△△△\n")
    return answer

def play_rv2(no, text):
    embed, corpus, cate = menu(no)
    list_return(text, embed, corpus, cate)

In [35]:
def list_return2(text, embed, corpus, cate):
    global answer
    answer = []
    query= text # str(input('검색하기 : '))
    cate_embedding = embed
    # 유사도 평가를 위한 작업
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    cos_scores = util.pytorch_cos_sim(query_embedding, cate_embedding)[0]
    cos_scores = cos_scores.cpu()
    # 상위 결과 추출
    top_results = np.argpartition(-cos_scores, range(5))[0:5]
    print("\n▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽\n")
    print("요청 문장:", query)
    print(f"🌼{cate} 카테고리의 리뷰들 중 가장 유사한 5개 리뷰문장🌼\n")
    for v, idx in enumerate(top_results[0:5]):
        text = corpus[idx].strip() 
        for i in range(3344):
            if text in df['rv'][i]:
                pname = df['pname'][i]
                x = [v+1, pname, text, np.round(cos_scores[idx],4)]
                answer.append(x)
#         print(v+1, pname, '\n','▶',text, "(Score: %.4f)" % (cos_scores[idx]))
#     print("\n△△△△△△△△△△△△△△△△△△\n")
    return answer

def play_rv3(no, text):
    embed, corpus, cate = menu(no)
    answer = list_return2(text, embed, corpus, cate)
    return answer

In [32]:
no = int(input('category'))
text = str(input('search_review:'))
answer = play_rv3(no, text)
print(answer)

category3
search_review:4호
------------------------------
★원하는 카테고리를 선택해주세요★
1. 전체
2. 기초
3. 색조
------------------------------

▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽

요청 문장: 4호
🌼색조 카테고리의 리뷰들 중 가장 유사한 5개 리뷰문장🌼

1 [NEW] 컬러그램 올인원 애교살 메이커 5종 
 ▶ 4번이 너무 예뻐요..🤓🧡✨✨ (Score: 0.6303)
2 [2022어워즈] 웨이크메이크 워터 블러링 픽싱 틴트 10colors 
 ▶ ✰ 지속력 4점 (Score: 0.5935)
3 [레오제이 PICK] 퓌 쿠션 글래스 리필기획(본품+리필+퍼프증정) 
 ▶ 퍼프가 4개 네요???? (Score: 0.5904)
4 [NEW] 롬앤 글래스팅 멜팅 밤 
 ▶ 있어보이는 4호 히피 베리와 6호 카야 피그를 (Score: 0.5552)
5 홀리카홀리카 마이페이브 무드 아이팔레트 
 ▶ 4번은 더 써본 팔레트인데요..😂 (Score: 0.5497)

△△△△△△△△△△△△△△△△△△

[[1, '[NEW] 컬러그램 올인원 애교살 메이커 5종', '4번이 너무 예뻐요..🤓🧡✨✨', tensor(0.6303)], [2, '[2022어워즈] 웨이크메이크 워터 블러링 픽싱 틴트 10colors', '✰ 지속력 4점', tensor(0.5935)], [3, '[레오제이 PICK] 퓌 쿠션 글래스 리필기획(본품+리필+퍼프증정)', '퍼프가 4개 네요????', tensor(0.5904)], [4, '[NEW] 롬앤 글래스팅 멜팅 밤', '있어보이는 4호 히피 베리와 6호 카야 피그를', tensor(0.5552)], [5, '홀리카홀리카 마이페이브 무드 아이팔레트', '4번은 더 써본 팔레트인데요..😂', tensor(0.5497)], [5, '홀리카홀리카 마이페이브 무드 아이팔레트', '4번은 더 써본 팔레트인데요..😂', tensor(0.5497)]]


In [29]:
answer

In [ ]:
# def 원본(query, embed, corpus, cate):
#     query= text # str(input('검색하기 : '))cate_embedding = embed
#     # 유사도 평가를 위한 작업
#     query_embedding = embedder.encode(query, convert_to_tensor=True)
#     cos_scores = util.pytorch_cos_sim(query_embedding, cate_embedding)[0]
#     cos_scores = cos_scores.cpu()
#     # 상위 결과 추출
#     top_results = np.argpartition(-cos_scores, range(5))[0:5]
#     print("\n▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽\n")
#     print("요청 문장:", query)
#     print(f"🌼{cate} 카테고리의 리뷰들 중 가장 유사한 5개 리뷰문장🌼\n")
#     for v, idx in enumerate(top_results[0:5]):
#         text = corpus[idx].strip() 
#         for i in range(3344):
#             if text in df['rv'][i]:
#                 pname = df['pname'][i]
#         print(v+1, pname, '\n','▶',text, "(Score: %.4f)" % (cos_scores[idx]))
#     print("\n△△△△△△△△△△△△△△△△△△\n")

In [6]:
no = int(input('category'))
text = str(input('search_review:'))
play_rv(no, text)

category3
search_review:여드름성 피부에 좋은 로션
------------------------------
★원하는 카테고리를 선택해주세요★
1. 전체
2. 기초
3. 색조
------------------------------

▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽

요청 문장: 여드름성 피부에 좋은 로션
🌼색조 카테고리의 리뷰들 중 가장 유사한 5개 리뷰문장🌼

1 [한정기획]클리오 킬커버 더뉴 파운웨어 쿠션 기획(본품+리필+와인잔2P+마커펜) 
 ▶ (보습력 강한 토너 사용하는 것도 중요) (Score: 0.7599)
2 [어워즈기획]에스쁘아 프로테일러 비벨벳 커버쿠션(본품+리필+미니쿠션) 
 ▶ 오일리하지 않은 보습력 좋은 토너 사용 (Score: 0.7235)
3 [NEW] 클리오 킬커버 메쉬글로우 쿠션 기획 
 ▶ 수분크림처럼 부드럽게 발리는데 (Score: 0.7203)
4 바닐라코 커버리셔스 얼티밋 화이트 쿠션 본품 14g+리필 14g / 4 colors 
 ▶ 지성 피부분들이 쓰시면 좋을 것 같아요! (Score: 0.7077)
5 아이레놀 다크서클 크림 2종 택1 (자연스러운 커버/진한 다크서클 커버) 
 ▶ 자연스럽고 촉촉한 아이크림인데 (Score: 0.6961)

△△△△△△△△△△△△△△△△△△



In [7]:
no = int(input('category'))
text = str(input('search_review:'))
play_rv(no, text)

category3
search_review:발색력 좋은 틴트
------------------------------
★원하는 카테고리를 선택해주세요★
1. 전체
2. 기초
3. 색조
------------------------------

▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽

요청 문장: 발색력 좋은 틴트
🌼색조 카테고리의 리뷰들 중 가장 유사한 5개 리뷰문장🌼

1 페리페라 잉크무드글로이틴트 
 ▶ 💚 쌩얼에 바르기 좋은 틴트 (Score: 0.9202)
2 페리페라 잉크무드글로이틴트 
 ▶ 촉촉이 틴트치고 지속력도 좋아서 (Score: 0.8835)
3 [NEW] 웨이크메이크 워터 컬러링 퓨어 틴트 6color 
 ▶ ✓ 수분감있는 틴트에요! (Score: 0.8641)
4 [NEW] 웨이크메이크 워터 컬러링 퓨어 틴트 6color 
 ▶ ✓ 수분감있는 틴트에요! (Score: 0.8641)
5 [유튜버 공동개발] 바닐라코 글로우 베일 틴트 3.8g / 5colors 
 ▶ 촉촉이 틴트로 착색도 살짝 있음. (Score: 0.8561)

△△△△△△△△△△△△△△△△△△



In [18]:
no = int(input('category'))
text = str(input('search_review:'))
answer = play_rv2(no, text)

category3
search_review:립스틱
------------------------------
★원하는 카테고리를 선택해주세요★
1. 전체
2. 기초
3. 색조
------------------------------

▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽

요청 문장: 립스틱
🌼색조 카테고리의 리뷰들 중 가장 유사한 5개 리뷰문장🌼

[[1, '3CE 소프트 매트 립스틱', '🍋  립스틱', tensor(0.9379)]]
1 3CE 소프트 매트 립스틱 
 ▶ 🍋  립스틱 (Score: 0.9379)
[[1, '3CE 소프트 매트 립스틱', '🍋  립스틱', tensor(0.9379)], [2, '[NEW] 컬러그램 올인원 오버립 메이커 2종', '립스틱을 이렇게', tensor(0.8671)]]
2 [NEW] 컬러그램 올인원 오버립 메이커 2종 
 ▶ 립스틱을 이렇게 (Score: 0.8671)
[[1, '3CE 소프트 매트 립스틱', '🍋  립스틱', tensor(0.9379)], [2, '[NEW] 컬러그램 올인원 오버립 메이커 2종', '립스틱을 이렇게', tensor(0.8671)], [3, '[NEW] 페리페라 잉크더벨벳AD 기상청컬렉션 컬러추가', '*립브러쉬를 사용해준다!', tensor(0.8086)]]
3 [NEW] 페리페라 잉크더벨벳AD 기상청컬렉션 컬러추가 
 ▶ *립브러쉬를 사용해준다! (Score: 0.8086)
[[1, '3CE 소프트 매트 립스틱', '🍋  립스틱', tensor(0.9379)], [2, '[NEW] 컬러그램 올인원 오버립 메이커 2종', '립스틱을 이렇게', tensor(0.8671)], [3, '[NEW] 페리페라 잉크더벨벳AD 기상청컬렉션 컬러추가', '*립브러쉬를 사용해준다!', tensor(0.8086)], [4, '[NEW] 페리페라 잉크더벨벳AD 기상청컬렉션 컬러추가', '립브러쉬 받았고', tensor(0.7995)]]
4 [NEW] 페리페라 잉크더벨벳AD 기상청컬렉션 컬러추가 
 ▶ 립브러쉬 

In [24]:
print(answer)

None


In [22]:
no = int(input('category'))
text = str(input('search_review:'))
answer = play_rv2(no, text)

category3
search_review:발색력 좋은 립스틱
------------------------------
★원하는 카테고리를 선택해주세요★
1. 전체
2. 기초
3. 색조
------------------------------

▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽

요청 문장: 발색력 좋은 립스틱
🌼색조 카테고리의 리뷰들 중 가장 유사한 5개 리뷰문장🌼

1 [NEW] 롬앤 글래스팅 멜팅 밤 
 ▶ 립밤같지 않게 발색이 참 좋아요 (Score: 0.8378)
2 [NEW] 컬러그램 올인원 오버립 메이커 2종 
 ▶ 립스틱을 이렇게 (Score: 0.8328)
3 [NEW] 무지개맨션 오브제 리퀴드 
 ▶ 발색력👍👍 (Score: 0.8307)
4 [NEW] 무지개맨션 오브제 리퀴드 
 ▶ 발색력👍👍 (Score: 0.8307)
5 페리페라 잉크무드 매트스틱 
 ▶ 👉👉 제 얼굴에는 립스틱만 바르면 이상해서 립스틱 바른 후에 촉촉한 틴트 발라주면 발색 완전 예쁨 (Score: 0.8300)

△△△△△△△△△△△△△△△△△△



In [23]:
answer